In [77]:
import pandas as pd
import os

In [78]:
os.chdir('/Users/daraklimova/Documents/Finance Project')

print(os.getcwd())

/Users/daraklimova/Documents/Finance Project


In [79]:
conda install xlrd > /dev/null 2>&1


Note: you may need to restart the kernel to use updated packages.


## Transforming the data bases

In [81]:
files_to_process = [
    "STMicroelectronics.xls",
    "Infineon.xls",
    "NXP Semiconductors .xls",
    "Melexis.xls",
    "SOITEC.xls",
    "Renesas.xls"
]

In [82]:
company_names = {
    "STMicroelectronics.xls": "STMicroelectronics",
    "Infineon.xls": "Infineon",
    "NXP Semiconductors .xls": "NXP Semiconductors",
    "Melexis.xls": "Melexis",
    "SOITEC.xls": "SOITEC",
    "Renesas.xls": "Renesas"
}

In [83]:
sheet_names = {
    "STMicroelectronics.xls": "STMPA",
    "Infineon.xls": "STMPA",
    "NXP Semiconductors .xls": "IFX",
    "Melexis.xls": "VNX",
    "SOITEC.xls": "MELE",
    "Renesas.xls": "SOI"
}

In [84]:
all_data = []

for file in files_to_process:
    try:
        df = pd.read_excel(file, sheet_name=sheet_names[file])
        df = df.dropna(how='all')
        df = df.set_index('Income Statement As of Originally Reported').T.reset_index()
        df = df.rename(columns={'index': 'date'})
        df['company_name'] = company_names[file]
        cols = df.columns.tolist()
        cols = ['company_name', 'date'] + [col for col in cols if col not in ['company_name', 'date']]
        df = df[cols]
        all_data.append(df)
        print(f"File {file} processed successfully")
    except Exception as e:
        print(f"Error processing file {file}: {str(e)}")

File STMicroelectronics.xls processed successfully
File Infineon.xls processed successfully
File NXP Semiconductors .xls processed successfully
File Melexis.xls processed successfully
File SOITEC.xls processed successfully
File Renesas.xls processed successfully


In [85]:
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df.to_csv("all_companies_combined.csv", index=False)
    print("\nAll files have been successfully processed and combined into all_companies_combined.csv")
    print("\nFirst 5 rows of the combined table:")
    print(combined_df.head())
else:
    print("No files were processed successfully")


All files have been successfully processed and combined into all_companies_combined.csv

First 5 rows of the combined table:
Income Statement As of Originally Reported        company_name  date  \
0                                           STMicroelectronics  2015   
1                                           STMicroelectronics  2016   
2                                           STMicroelectronics  2017   
3                                           STMicroelectronics  2018   
4                                           STMicroelectronics  2019   

Income Statement As of Originally Reported       Revenue Revenue Growth %  \
0                                             6900000000            -6.85   
1                                             6970000000             1.10   
2                                             8350000000            19.70   
3                                           9660000000.0            15.78   
4                                           9560000000.0

In [86]:
#Adding the financial health ratios
COMPANY_FILES = [
    "STMicroelectornics_financial.xls",
    "Infineon_financial.xls",
    "NXP Semiconductors_financial.xls",
    "Melexis_financial.xls",
    "SOITEC_financial.xls",
    "Renesas_financial.xls"
]

COMPANY_MAPPING = {
    "STMicroelectornics_financial.xls": "STMicroelectronics",
    "Infineon_financial.xls": "Infineon",
    "NXP Semiconductors_financial.xls": "NXP Semiconductors",
    "Melexis_financial.xls": "Melexis",
    "SOITEC_financial.xls": "SOITEC",
    "Renesas_financial.xls": "Renesas"
}

SHEET_MAPPING = {
    "STMicroelectornics_financial.xls": "STMPA",
    "Infineon_financial.xls": "IFX",
    "NXP Semiconductors_financial.xls": "VNX",
    "Melexis_financial.xls": "MELE",
    "SOITEC_financial.xls": "SOI",
    "Renesas_financial.xls": "NEN"
}

financial_metrics_records = []

for financial_file in COMPANY_FILES:
    try:
        company_data = pd.read_excel(financial_file, sheet_name=SHEET_MAPPING[financial_file])
        company_data = company_data.dropna(how='all')
        company_data = company_data.set_index(company_data.columns[0])
        company_data = company_data.T.reset_index()
        company_data = company_data.rename(columns={'index': 'fiscal_period'})
        company_data['company_name'] = COMPANY_MAPPING[financial_file]
        column_order = ['company_name', 'fiscal_period'] + [
            col for col in company_data.columns 
            if col not in ['company_name', 'fiscal_period']
        ]
        company_data = company_data[column_order]
        financial_metrics_records.append(company_data)
        print(f"Processed {financial_file} successfully")
    except Exception as error:
        print(f"Error processing {financial_file}: {str(error)}")

if financial_metrics_records:
    consolidated_financial_metrics = pd.concat(financial_metrics_records, ignore_index=True)
    output_filename = "consolidated_financial_health_metrics.csv"
    consolidated_financial_metrics.to_csv(output_filename, index=False)
    print(f"\nSuccessfully created {output_filename} with financial health metrics")
    print("\nPreview of consolidated data:")
    print(consolidated_financial_metrics.head())
else:
    print("No financial data was processed successfully")

Processed STMicroelectornics_financial.xls successfully
Processed Infineon_financial.xls successfully
Processed NXP Semiconductors_financial.xls successfully
Processed Melexis_financial.xls successfully
Processed SOITEC_financial.xls successfully
Processed Renesas_financial.xls successfully

Successfully created consolidated_financial_health_metrics.csv with financial health metrics

Preview of consolidated data:
Liquidity/Financial Health        company_name fiscal_period Current Ratio  \
0                           STMicroelectronics       2015-12           3.0   
1                           STMicroelectronics       2016-12          2.82   
2                           STMicroelectronics       2017-12          2.52   
3                           STMicroelectronics       2018-12          2.76   
4                           STMicroelectronics       2019-12          3.03   

Liquidity/Financial Health Quick Ratio Interest Coverage Financial Leverage  \
0                                 2

## Cleaning the datasets and merging them

In [88]:
#Cleaning data from the main dataset
cleaned_df = combined_df[combined_df['date'] != 'TTM'].copy()

print(cleaned_df.head())

Income Statement As of Originally Reported        company_name  date  \
0                                           STMicroelectronics  2015   
1                                           STMicroelectronics  2016   
2                                           STMicroelectronics  2017   
3                                           STMicroelectronics  2018   
4                                           STMicroelectronics  2019   

Income Statement As of Originally Reported       Revenue Revenue Growth %  \
0                                             6900000000            -6.85   
1                                             6970000000             1.10   
2                                             8350000000            19.70   
3                                           9660000000.0            15.78   
4                                           9560000000.0            -1.12   

Income Statement As of Originally Reported  Gross Profit  \
0                                           

In [89]:
#Cleaning the data from the second dataset
# 1. Convert fiscal_period from YYYY-MM to YYYY
consolidated_financial_metrics['fiscal_period'] = (
    consolidated_financial_metrics['fiscal_period']
    .astype(str)  # Ensure it's string type
    .str[:4]  # Take first 4 characters (the year)
)

# 2. Remove temporary 'year' column if it exists
if 'year' in consolidated_financial_metrics.columns:
    consolidated_financial_metrics = consolidated_financial_metrics.drop(columns=['year'])

# 3. Remove rows containing 'Late' in any column
# First convert all data to strings for comparison
str_data = consolidated_financial_metrics.astype(str)
# Create mask of rows that don't contain 'Late'
no_late_mask = ~str_data.apply(lambda x: x.str.contains('Late', case=False)).any(axis=1)
# Apply the filter
consolidated_financial_metrics = consolidated_financial_metrics[no_late_mask]

# Save the cleaned data
output_filename = "finished_financial_health_metrics.csv"
consolidated_financial_metrics.to_csv(output_filename, index=False)

print(consolidated_financial_metrics.head())

Liquidity/Financial Health        company_name fiscal_period Current Ratio  \
0                           STMicroelectronics          2015           3.0   
1                           STMicroelectronics          2016          2.82   
2                           STMicroelectronics          2017          2.52   
3                           STMicroelectronics          2018          2.76   
4                           STMicroelectronics          2019          3.03   

Liquidity/Financial Health Quick Ratio Interest Coverage Financial Leverage  \
0                                 2.05              3.23               1.77   
1                                 1.96              6.03               1.77   
2                                 1.79             19.33               1.79   
3                                 1.93             26.72               1.71   
4                                 2.14             23.02               1.69   

Liquidity/Financial Health Debt/Equity Book Value/Share 

In [90]:
#Merging datasets
# Find common columns for merging (company name and year)
common_columns = ['company_name']
date_col_cleaned = 'date'  # Adjust if different in cleaned_df
date_col_financial = 'fiscal_period'  # Adjust if different in financial metrics

# Extract year from both date columns
cleaned_df['merge_year'] = pd.to_datetime(cleaned_df[date_col_cleaned]).dt.year.astype(str)
consolidated_financial_metrics['merge_year'] = consolidated_financial_metrics[date_col_financial].str[:4]

# Perform left join
merged_data = pd.merge(
    cleaned_df,
    consolidated_financial_metrics.drop(columns=[date_col_financial]),  # Remove original date column
    left_on=['company_name', 'merge_year'],
    right_on=['company_name', 'merge_year'],
    how='left'
)

# Clean up temporary columns
merged_data = merged_data.drop(columns=['merge_year'])

# Save merged data
merged_data.to_csv("complete_merged_dataset.csv", index=False)

print(merged_data.head(3).to_markdown(index=False))

| company_name       |   date |    Revenue |   Revenue Growth % |   Gross Profit |   Gross Profit Margin % |   Operating Income |   Operating Margin % |      EBIT |   EBIT Margin % |     EBITDA |   EBITDA Margin % |   Net Income |   Net Profit Margin % |   Basic EPS |   Diluted EPS | Normalized EPS   |   Dividend Per Share |   Balance Sheet As of Originally Reported |   Total Assets |   Total Liabilities |   Total Debt |   Total Equity |   Cash And Cash Equivalents |   Working Capital |   Shares Outstanding Capital |   Book Value per Share |   Debt to Equity |   Cash Flow As of Originally Reported |   Cash From Operating Activities |   Cash From Investing Activities |   Cash From Financing Activities |   Capital Expenditure |   Free Cash Flow |   Change in Cash |   Current Ratio |   Quick Ratio |   Interest Coverage |   Financial Leverage |   Debt/Equity |   Book Value/Share |   Cap Ex as a % of Sales |
|:-------------------|-------:|-----------:|-------------------:|---------------:|-

In [91]:
#Extracting the essential data
essential_columns = [
    'company_name',
    'date',
    'Revenue',
    'Gross Profit',
    'Net Income',
    'Basic EPS',
    'Gross Profit Margin %',
    'Net Profit Margin %',
    'Cash And Cash Equivalents',
    'Total Assets',
    'Total Liabilities',
    'Total Debt',
    'Total Equity',
    'Shares Outstanding Capital',
    'Current Ratio',         
    'Quick Ratio',
]
    
filtered_df = merged_data[essential_columns]

print(filtered_df)

filtered_df.to_csv('filtered_financial_data.csv', index=False)

          company_name  date        Revenue  Gross Profit    Net Income  \
0   STMicroelectronics  2015     6900000000    2330000000     100000000   
1   STMicroelectronics  2016     6970000000    2460000000     170000000   
2   STMicroelectronics  2017     8350000000    3270000000     800000000   
3   STMicroelectronics  2018   9660000000.0  3860000000.0  1290000000.0   
4   STMicroelectronics  2019   9560000000.0  3700000000.0  1030000000.0   
5   STMicroelectronics  2020  10220000000.0  3790000000.0  1110000000.0   
6   STMicroelectronics  2021  12760000000.0  5330000000.0  2000000000.0   
7   STMicroelectronics  2022  16130000000.0  7640000000.0  3960000000.0   
8   STMicroelectronics  2023  17290000000.0  8290000000.0  4210000000.0   
9   STMicroelectronics  2024  13270000000.0  5220000000.0  1560000000.0   
10            Infineon  2015     6900000000    2330000000     100000000   
11            Infineon  2016     6970000000    2460000000     170000000   
12            Infineon  2

In [92]:
melted_df = filtered_df.melt(id_vars=['company_name', 'date'], 
                           var_name='metric', 
                           value_name='value')

# Pivot the table to get metrics as columns
transformed_df = melted_df.pivot(index=['date', 'company_name'], 
                               columns='metric', 
                               values='value').reset_index()

# Rename columns and sort by year
transformed_df = transformed_df.rename(columns={'date': 'Year', 'company_name': 'Company'})
transformed_df = transformed_df.sort_values('Year')

# Reset index
transformed_df.reset_index(drop=True, inplace=True)

# Save both versions
filtered_df.to_csv('filtered_financial_data.csv', index=False)
transformed_df.to_csv('transformed_financial_data.csv', index=False)

In [109]:
# Convert all columns except 'Company' to numeric
cols_to_convert = [col for col in transformed_df.columns if col != 'Company']
transformed_df[cols_to_convert] = transformed_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Remove rows before 2015 and reset index
transformed_df = transformed_df[transformed_df['Year'] >= 2015].reset_index(drop=True)

# Save to new CSV
transformed_df.to_csv('ready_cleaned_data.csv', index=False)

# Verification
print(f"Saved {len(transformed_df)} rows to final_cleaned_data.csv")
print("First 3 rows:")
print(transformed_df.head(3))

Saved 57 rows to final_cleaned_data.csv
First 3 rows:
metric  Year             Company  Basic EPS  Cash And Cash Equivalents  \
0       2015            Infineon       0.12               1.770000e+09   
1       2015             Melexis       6.36               1.610000e+09   
2       2015  NXP Semiconductors       0.56               6.700000e+08   

metric  Current Ratio  Gross Profit  Gross Profit Margin %    Net Income  \
0                2.60  2.330000e+09                  33.81  1.000000e+08   
1                3.67  2.790000e+09                  45.68  1.530000e+09   
2                1.89  2.080000e+09                  35.89  6.300000e+08   

metric  Net Profit Margin %  Quick Ratio       Revenue  \
0                      1.59         1.77  6.900000e+09   
1                     26.21         2.42  6.100000e+09   
2                     10.73         1.08  5.800000e+09   

metric  Shares Outstanding Capital  Total Assets    Total Debt  Total Equity  \
0                     8.800000e